# Twitter API

We will learn how to extract data from Twitter API.

Tweepy is easy-to-use Python library for accessing the API.

Here is a link for tweepy documentation, https://tweepy.readthedocs.io/en/latest/index.html

# Prerequisite
create a twitter account, if you already have it, please skip this step. 

In [ ]:
#install some dependecies
!pip install tweepy
!pip install wordcloud
!pip install textblob
!pip install nltk

In [ ]:
#import the libraries
import tweepy as tp
import json
import pandas as pd

In [ ]:
#Authentication 
consumer_key = 'cYo1ZjVZtWujP5mJd56Ej6oPc'
consumer_secret = 'SyCp8W7yOi30NasRv7hFSQwI10ts1Df9NW5hDbGSsrhqqIhXkM'
access_token = '1294698007-hkhnPycvlvwA5TqUfFCz64DM5yFCoRxJqarZKFU'
access_token_secret = 'V3vgOUJD5SjGlxyl6qzbQUvpqoNcB40oGYBtFHkAKV4wX'

auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tp.API(auth)

# 1. Obtain data by keyword or user (Search API)

We use api.search to find relevant Tweets based on our search

In [ ]:
#get tweet 
#q is the query, lang is language, items is how many tweets
#q can be username, hashtag, keyword
tweets = tp.Cursor(api.search, q="trianonurhikmat", lang="id").items(5)

#store result in a list
result = []
for tweet in tweets :
    result.append(tweet)

In [ ]:
#print how many tweets and type of result
print(type(result))
print(len(result))

#get tweet text
for tweet in result :
    print(tweet.text)

In [ ]:
# extract tweet to the pandas dataframe
def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    df = pd.DataFrame(id_list, columns = ["id"])
    
    df["text"] = [tweet.text for tweet in results]
    df["created_at"] = [tweet.created_at for tweet in results]
    df["retweet_count"] = [tweet.retweet_count for tweet in results]
    df["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    df["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    df["user_location"] = [tweet.author.location for tweet in results]
    df["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]

    return df

In [ ]:
# use the function to get dataframe of the tweet
df = tweets_df(result)
df.head()

# 2. Find the trend topics (Trends API)

Twitter provides global trends as well as localized tweets.

In [ ]:
#get the detail of countries
world_trends = api.trends_available()
world_trends = pd.DataFrame(world_trends)
print(world_trends)

In [ ]:
#find the wanted country
ind = world_trends.loc[world_trends['country'] == 'Indonesia']
ind

In [ ]:
# obtain trend in one country based on the woeid
ind_trends = api.trends_place(id = 23424846)
trends = json.loads(json.dumps(ind_trends, indent = 1))

trend_list = []
#store into data frame
for index, trend in enumerate(trends[0]["trends"]):
    trend_list.append((index + 1, trend["name"].strip('#')))

df = pd.DataFrame(trend_list, columns = ['id', 'name'])
df